In [ ]:
!wget https://zenodo.org/records/5193932/files/covidbr_labeled.csv -O data/raw/COVID19BR.csv
!wget https://github.com/MuMiN-dataset/mumin-baseline/raw/main/claim_dump_large.pkl.xz -O data/raw/claim_dump_large.pkl.xz
!unxz data/raw/"claim_dump_large.pkl.xz"
!git clone https://github.com/roneysco/Fake.br-Corpus data/src/

In [1]:
import pickle
import time
import os
import re

from lingua import Language, LanguageDetectorBuilder
from urlextract import URLExtract
from datasets import load_dataset
from akin import MinHash, LSH
from tqdm.auto import tqdm
import pandas as pd
import tiktoken
import demoji

from utils import clean_text, remove_punc

tqdm.pandas()
extractor = URLExtract()
detector = LanguageDetectorBuilder.from_all_languages().build()
encoder = tiktoken.get_encoding("cl100k_base")
data = dict()

In [ ]:
data["COVID19.BR"] = pd.read_csv("data/raw/COVID19BR.csv")
data["COVID19.BR"]["text"] = data["COVID19.BR"]["text"].str.strip()
data["COVID19.BR"].rename(columns={"misinformation":"label"}, inplace=True)
data["COVID19.BR"]["label"] = data["COVID19.BR"]["label"].map({1:"fake", 0:"true"})
data["COVID19.BR"]["metadata"] = data["COVID19.BR"].apply(lambda row: {"revision": row["revision"], "shares": row["shares"], "source": row["source"]}, axis=True)
data["COVID19.BR"].drop(columns=["revision", "shares", "source"], inplace=True)
data["COVID19.BR"] = data["COVID19.BR"].reset_index(names="id")

In [ ]:
with open("data/raw/claim_dump_large.pkl", "rb") as f:
  data["MuMiN-PT"] = pickle.load(f)

data["MuMiN-PT"]["claim"] = data["MuMiN-PT"]["claim"].str.strip()
pt_filter = data["MuMiN-PT"]["claim"].progress_apply(lambda t: detector.detect_language_of(t) == Language.PORTUGUESE)
data["MuMiN-PT"] = data["MuMiN-PT"][pt_filter].reset_index(names="id")
data["MuMiN-PT"].to_parquet('data/parquet/MuMiN-PT.parquet')

data["MuMiN-PT"]["split"] = data["MuMiN-PT"].apply(lambda row: "train" if row.train_mask else "dev" if row.val_mask else "test", axis=True)
data["MuMiN-PT"]["metadata"] = data["MuMiN-PT"].apply(lambda row: {"text_en": row.claim_en}, axis=True)
data["MuMiN-PT"].drop(columns=["train_mask", "val_mask", "test_mask", "claim_en"], inplace=True)
data["MuMiN-PT"].rename(columns={"claim": "text", "verdict": "label"}, inplace=True)
data["MuMiN-PT"]["label"] = data["MuMiN-PT"]["label"].map({"misinformation": "fake", "factual":"true"})

  0%|          | 0/13205 [00:00<?, ?it/s]

In [ ]:
config_dir = "data/src/Fake.br-Corpus/"

_METADATA_COLS = [
    "author",
    "link",
    "category",
    "date of publication",
    "number of tokens",
    "number of words without punctuation",
    "number of types",
    "number of links inside the news",
    "number of words in upper case",
    "number of verbs",
    "number of subjuntive and imperative verbs",
    "number of nouns",
    "number of adjectives",
    "number of adverbs",
    "number of modal verbs (mainly auxiliary verbs)",
    "number of singular first and second personal pronouns",
    "number of plural first personal pronouns",
    "number of pronouns",
    "pausality",
    "number of characters",
    "average sentence length",
    "average word length",
    "percentage of news with speeling errors",
    "emotiveness",
    "diversity",
]

data["Fake.br"] = list()
for label in ["fake", "true"]:
    label_dir = os.path.join(config_dir, "size_normalized_texts", label)

    for example in tqdm(os.listdir(label_dir)):
        example_path = os.path.join(label_dir, example)

        with open(example_path, "r") as f:
            text = f.read()

        pair_idx = example.replace(".txt", "")

        row = {
            "id": f"{label}_{int(pair_idx):04}",
            "text": text,
            "label": label
        }

        
        metadata_path = os.path.join(
            config_dir,
            "full_texts",
            f"{label}-meta-information",
            example.replace(".txt", "-meta.txt"),
        )

        if os.path.exists(metadata_path): # https://github.com/roneysco/Fake.br-Corpus/issues/7
            with open(metadata_path, "r") as f:
                metadata = f.read().split("\n")
            metadata = dict(zip(_METADATA_COLS, metadata))
            row["date"] = metadata.pop("date of publication")

            example_path = os.path.join(config_dir, "full_texts", label, example)
            with open(example_path, "r") as f:
                metadata["full_text"] = f.read()
        else:
            metadata = dict()
   
        row["metadata"] = metadata
        data["Fake.br"].append(row)

data["Fake.br"] = pd.DataFrame(data["Fake.br"])

data["Fake.br"]["text"] = data["Fake.br"]["text"].str.strip()

months = [
    "janeiro",
    "fevereiro",
    "março",
    "abril",
    "maio",
    "junho",
    "julho",
    "agosto",
    "setembro",
    "outubro",
    "novembro",
    "dezembro"
]


date = data['Fake.br']["date"]
date = date.str.replace(r"\d\dh\d\d", "", regex=True)
date = date.str.strip()

for i, month in enumerate(months):
    date = date.str.replace(f" de {month} de ", f"/{i+1}/")

date = date.str.replace(r"(\d\d)/(\d\d)/(\d\d\d\d)", r"\3-\2-\1", regex=True)
date = date.str.replace(r"(\d)/(\d\d)/(\d\d\d\d)", r"\3-\2-0\1", regex=True)
date = date.str.replace(r"(\d\d)/(\d)/(\d\d\d\d)", r"\3-0\2-\1", regex=True)
date = date.str.replace(r"(\d)/(\d)/(\d\d\d\d)", r"\3-0\2-0\1", regex=True)
date = date.str.replace("28/2/18", "2018-02-28")
date = date.str.strip()
date.loc[2891] = "2017-09-03"
date.loc[3613] = "2017-12-04"
date.loc[4300] = "2018-01-06"
data['Fake.br']["date"] = pd.to_datetime(date, format="%Y-%m-%d")
data["Fake.br"]["metadata"] = data["Fake.br"].apply(
    lambda row: row["metadata"] | {"date of publication": row["date"]}, axis=True # python3.9
)
data["Fake.br"].drop(columns="date", inplace=True)

  0%|          | 0/3600 [00:00<?, ?it/s]

  0%|          | 0/3600 [00:00<?, ?it/s]

In [5]:
for dataset in data.keys():
  data[dataset]["text_urls"] = data[dataset]["text"].progress_apply(lambda t: [] if pd.isna(t) else extractor.find_urls(t))

def remove_url(row):
    text = row["text"]
    
    for url in row.text_urls:
        text = text.replace(url, "")

    if pd.isna(text):
        return text

    text = text.strip()

    return text

for dataset in data:
    data[dataset]["text_no_url"] = data[dataset].apply(remove_url, axis=True)
    data[dataset]["only_url"] = data[dataset]["text_no_url"].apply(lambda t: len(t) < 2 if pd.notna(t) else False)

  0%|          | 0/2899 [00:00<?, ?it/s]

  0%|          | 0/1404 [00:00<?, ?it/s]

  0%|          | 0/7200 [00:00<?, ?it/s]

Remover exemplos:
 - Duplicados
 - Com menos de X1 tokens ou mais de X2 (OpenAI via tiktoken) considerar emojis, removendo pontuação e outras limpezas
 - Termos não relacionados
 - URLs puras

In [6]:
# https://github.com/roneysco/Fake.br-Corpus/issues/7
data["Fake.br"]["no_full_text"] = data["Fake.br"]["metadata"].apply(lambda d: d.get("date of publication")).isna()

for dataset in data.keys():
  data[dataset]["duplicated"] = data[dataset].duplicated(subset="text")
  data[dataset]["null"] = data[dataset]["text"].isna()
data["Fake.br"]["url_duplicated"] = data["Fake.br"]["metadata"].apply(
   lambda u: u['link'] if "link" in u else time.time()).duplicated() # não considerar os nulos como duplicatas

unrelated_content = [
    "SEX DATING",
    'Acesse este link para entrar no meu grupo do WhatsApp',
    "grupos adulto", # multiplas claims
    "Notícias do Dia - Ano", # multiplas claims
    "Boletim da noite", # multiplas claims
    "Boletim da Manhã", # multiplas claims
    "Confira os destaques do AssisCity", #multiplas claims
    "Resumo do dia", # multiplas claims
    "🧙🏻♀HORÓSCOPO🧙🏻",
    "ĆŘ€ƗØ ΩỮ€ Δ ΜØŘŦ€ Ř€ŞØŁVΔ ŦỮĐØ €ŇŦŘ€Ɨ ƤŘΔ ŞØΜΔŘ",
    "dia do ano no calendário gregoriano"
]

unrelated_content = r"|".join(unrelated_content)
data['COVID19.BR']["unrelated_content"] = data['COVID19.BR']["text_no_url"].str.contains(unrelated_content, regex=True)

n_tokens_covid = data['COVID19.BR']["text_no_url"].progress_apply(
    lambda t: 0 if pd.isna(t) else len(encoder.encode(demoji.replace(clean_text(remove_punc(t))))))    
data['COVID19.BR']["few_tokens"] = n_tokens_covid <= 15

contradicts = [ 
    [242, 1423],
    [469, 1721],
    [910, 2741],
    [88, 274, 995],
    [1639, 2071, 2698],
    [336, 1184, 2728],
    [388,430, 1930, 2199, 2680]
]

contradicts = [item for _ in contradicts for item in _]
data["COVID19.BR"]["contradicts"] = data["COVID19.BR"].apply(lambda row: row.name in contradicts, axis=True)

not_pt = [1167, 1495, 1497, 1565, 1621, 2542, 2768, 2444]

data["COVID19.BR"]["not_pt"] = data["COVID19.BR"].apply(lambda row: row.name in not_pt, axis=True)

multiple_claims = data["COVID19.BR"][data["COVID19.BR"]["text"].notna() & data["COVID19.BR"]["text"].str.contains("principais notícias", flags=re.IGNORECASE)].index.to_list()
opinion = [
    [117, 254, 393, 421, 434, 1438, 2229, 2499],
    [67, 217, 269, 1706, 2379, 2380, 2562, 2592, 2793]
]
  
unrelated = [item for _ in opinion for item in _]  + multiple_claims

data["COVID19.BR"]["unrelated_content"] = data["COVID19.BR"].apply(lambda row: row.unrelated_content or row.name in unrelated, axis=True)
data["COVID19.BR"]["manual_review"] = data["COVID19.BR"].apply(lambda r: pd.notna(r["text"]) and ("fraude paulista" in r["text"].lower() and r["label"] == "true"), axis=True)

remove_idx = data[dataset].columns.to_list().index("text_no_url") + 1
to_be_removed = data["Fake.br"][data["Fake.br"][data["Fake.br"].columns[remove_idx:]].any(axis=True)]["id"]
idxs = {idx.split("_")[-1] for idx in to_be_removed}

data["Fake.br"]["label_align"] = data["Fake.br"]["id"].apply(lambda idx: idx.split("_")[-1] in idxs and idx not in to_be_removed)

  0%|          | 0/2899 [00:00<?, ?it/s]

In [ ]:
data["Fake.br"].sort_values("id",key=lambda idx: idx.str.replace(r"(.*)_(.*)", r"\2_\1", regex=True), inplace=True)

raw_data = dict()

for dataset in data:
    if data[dataset].index.name != "id":
        data[dataset] = data[dataset].set_index("id")
    data[dataset].to_parquet(f"data/parquet/original/{dataset}.parquet")
    raw_data[dataset] = data[dataset].copy()

data["COVID19.BR"]["label"] = data["COVID19.BR"].apply(lambda r: "fake" if pd.notna(r["text"]) and "fraude paulista" in r["text"].lower() else r["label"], axis=True)

In [9]:
for dataset in tqdm(data):
    remove_idx = data[dataset].columns.to_list().index("text_no_url") + 1
    remove_columns = data[dataset].columns[remove_idx:].to_list()

    if "manual_review" in remove_columns:
        remove_columns.remove("manual_review")

    data[dataset] = data[dataset][~data[dataset][remove_columns].any(axis=True)]
    data[dataset] = data[dataset][data[dataset].columns[:remove_idx]]

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def find_near_duplicates(df):
    minhash = MinHash(df["text_no_url"].to_list(), seed=3, n_gram=5, hash_bits=128)

    lsh = LSH(minhash, df.index.to_list(), no_of_bands=50)
    
    return lsh.adjacency_list(min_jaccard=0.7)

for dataset in tqdm(data):
    data[dataset]["near_duplicates"] = find_near_duplicates(data[dataset])

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
for dataset in data:
    data[dataset].to_parquet(f'data/parquet/prepro/{dataset}.parquet')

: 